In [90]:
import pandas as pd
import numpy as np
import gensim
from gsdmm import MovieGroupProcess
import re
import nltk
import string
from nltk.corpus import stopwords
import os
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import shutil
stopwords = stopwords.words('english')


In [91]:
df = pd.read_csv("../../data/extracted/TRAINING/training.csv", delimiter='\t')
print(df.shape)
df.head(3)

(10000, 7)


,file_name,misogynous,shaming,stereotype,objectification,violence,Text Transcription
0,1.jpg,0,0,0,0,0,Milk Milk.zip
1,10.jpg,1,0,0,0,1,"ROSES ARE RED, VIOLETS ARE BLUE IF YOU DON'T S..."
2,1000.jpg,0,0,0,0,0,BREAKING NEWS: Russia releases photo of DONALD...


In [92]:
def preprocess(text):
    text = text.lower()
    text = ' '.join([word for word in text.split() if word not in stopwords])
    text = ' '.join([word for word in text.split() if len(word) >= 3])
    text = ' '.join([word for word in text.split() if not re.match(r'\b\w+\.(com|org|net)\b', word)])
    text = text.translate(str.maketrans('', '', string.punctuation))
    return text

texts = df['Text Transcription'].tolist()
texts = list(map(preprocess, texts))

In [93]:
num_topics = 6

In [94]:
docs = [text.split() for text in texts]

dictionary = gensim.corpora.Dictionary(docs)

dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

vocab_length = len(dictionary)

bow_corpus = [dictionary.doc2bow(doc) for doc in docs]

gsdmm = MovieGroupProcess(K=num_topics, alpha=0.1, beta=0.3, n_iters=15)

y = gsdmm.fit(docs, vocab_length)

In stage 0: transferred 7378 clusters with 6 clusters populated
In stage 1: transferred 4798 clusters with 6 clusters populated
In stage 2: transferred 3869 clusters with 6 clusters populated
In stage 3: transferred 3475 clusters with 6 clusters populated
In stage 4: transferred 3150 clusters with 6 clusters populated
In stage 5: transferred 3072 clusters with 6 clusters populated
In stage 6: transferred 2829 clusters with 6 clusters populated
In stage 7: transferred 2727 clusters with 6 clusters populated
In stage 8: transferred 2695 clusters with 6 clusters populated
In stage 9: transferred 2595 clusters with 6 clusters populated
In stage 10: transferred 2482 clusters with 6 clusters populated
In stage 11: transferred 2466 clusters with 6 clusters populated
In stage 12: transferred 2461 clusters with 6 clusters populated
In stage 13: transferred 2451 clusters with 6 clusters populated
In stage 14: transferred 2353 clusters with 6 clusters populated


In [95]:
doc_count = np.array(gsdmm.cluster_doc_count)
print('Number of documents per topic :', doc_count)

top_index = doc_count.argsort()[-num_topics:][::-1]
print('Most important clusters (by number of docs inside):', top_index)

# define function to get top words per topic
def top_words(cluster_word_distribution, top_cluster, values):
    for cluster in top_cluster:
        sort_dicts = sorted(cluster_word_distribution[cluster].items(), key=lambda k: k[1], reverse=True)[:values]
        print("\nCluster %s : %s"%(cluster, sort_dicts))


Number of documents per topic : [1547 2674 1726 1182 2211  660]
Most important clusters (by number of docs inside): [1 4 2 0 3 5]


In [96]:
path_to_font = "milky_coffee/Milky Coffee.ttf"


output_directory = "outputs"
shutil.rmtree(output_directory)
os.mkdir(output_directory)



cluster_word_distribution = gsdmm.cluster_word_distribution
topic_num = 0
num_words = 20

for topic_num in top_index:

    topic_dict = dict(sorted(cluster_word_distribution[topic_num].items(), key=lambda k: k[1], reverse=True)[:num_words])

    # Generate a word cloud image
    wordcloud = WordCloud(background_color='#fcf2ed', 
                                width=1800,
                                height=700,
                                font_path=path_to_font,
                                colormap='flag').generate_from_frequencies(topic_dict)

    wordcloud.to_file(os.path.join(output_directory, f"topic_{topic_num}.png"))

In [100]:
gsdmm.choose_best_label(doc = ['feminist', 'cooking', 'kitchen'])

(4, 0.40652958046338517)